<a href="https://colab.research.google.com/github/yuli4ka/ReversiGene/blob/main/Reversi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
boardSize = 6
w = 1 #white player
b = -1 #black player
white = "☺"
black = "☻"
zero = "."

In [118]:
def getSymbol(number):
  if number == w:
    return white
  elif number == b:
    return black
  else:
    return zero

In [119]:
def chessCount(board, color):
  return sum(board[i].count(color) for i in range(boardSize))

In [120]:
def printBoard(board):
  print()
  for i in range(boardSize):
    for j in range(boardSize):
      print(getSymbol(board[i][j]), end = " ")
    print()
  print()
  print(black + ":" + str(chessCount(board, b)))
  print(white + ":" + str(chessCount(board, w)))
  print()

In [121]:
def getStartBoard():
  board = [[0 for _ in range(boardSize)] for _ in range(boardSize)]
  board[boardSize // 2 - 1][boardSize // 2 - 1] = w
  board[boardSize // 2 - 1][boardSize // 2] = b
  board[boardSize // 2][boardSize // 2 - 1] = b
  board[boardSize // 2][boardSize // 2] = w
  return board 

In [122]:
def outOfBound(i, j):
  return i < 0 or i >= boardSize or j < 0 or j >= boardSize

In [123]:
def opposite(turn):
  return w if turn == b else b

In [124]:
def neighbourIsOpposite(board, i, j, di, dj, turn):
  if (outOfBound(i + di, j + dj)):
    return False
  return board[i + di][j + dj] == opposite(turn)

In [125]:
def checkSandwich(board, i, j, di, dj, turn):
  if di == 0 and dj == 0:
    return False
  if outOfBound(i + di, j + dj):
    return False
  if board[i + di][j + dj] == turn:
    return True
  if board[i + di][j + dj] == opposite(turn):
    return checkSandwich(board, i  + di, j + dj, di, dj, turn)
  return False

In [126]:
def turnIsValid(board, i, j, turn):
  if outOfBound(i, j):
    return False
  if board[i][j] != 0:
    return False
  for di in range (-1, 2):
    for dj in range(-1, 2):
      if neighbourIsOpposite(board, i, j, di, dj, turn):
        if checkSandwich(board, i, j, di, dj, turn):
          return True
  return False

In [127]:
def replaceNeighbours(board, i, j, di, dj, turn):
  if checkSandwich(board, i, j, di, dj, turn):
    board[i + di][j + dj] = turn
    replaceNeighbours(board, i + di, j + dj, di, dj, turn)

In [128]:
def getValidMoves(board, turn):
  return[(i, j) for i in range(boardSize) 
   for j in range(boardSize) 
    if turnIsValid(board, i, j, turn)]

In [129]:
def move(board, i, j, turn):
  board[i][j] = turn
  for di in range(-1, 2):
    for dj in range(-1, 2):
      if neighbourIsOpposite(board, i, j, di, dj, turn):
        replaceNeighbours(board, i, j, di, dj, turn)
  return board

In [130]:
board = getStartBoard()
printBoard(board)


. . . . . . 
. . . . . . 
. . ☺ ☻ . . 
. . ☻ ☺ . . 
. . . . . . 
. . . . . . 

☻:2
☺:2

